In [3]:
from __future__ import print_function

import numpy as np
import pandas as pd
np.random.seed(1)

import sys
import sklearn
import sklearn.ensemble
import sklearn.cluster
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

import shap
from anchor import utils
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality', discretize=False, balance=True, dataset_folder=dataset_folder)

In [ ]:
dataset2 = utils.load_dataset('mortality', discretize=True, balance=True, dataset_folder=dataset_folder)
#cat_names = sorted(dataset2.categorical_names.keys())   #useless
#n_values = [len(dataset2.categorical_names[i]) for i in cat_names]  # useless
#enc = OneHotEncoder(categorical_features=cat_names, n_values = n_values)  # useless
enc = OneHotEncoder()
enc.fit(dataset2.data[0:100])

In [5]:
X_train = pd.DataFrame(data=dataset.train, columns=dataset.feature_names)
y_train = dataset.labels_train
X_test = pd.DataFrame(data=dataset.test, columns=dataset.feature_names)
y_test = dataset.labels_test

In [6]:
shap.initjs()

In [7]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 1.0
Test 0.8504784688995215


In [ ]:
#explainer = shap.KernelExplainer(predict_fn, X_train)
#shap_values = explainer.shap_values(X_test)

In [134]:
#K = 100
#X_train_summary = shap.kmeans(X_train, K)
#explainer = shap.KernelExplainer(predict_fn, X_train_summary)

In [8]:
explainer = shap.KernelExplainer(predict_fn, X_train.median())

In [ ]:
#shap_values = explainer.shap_values(X_test)

In [149]:
def calc_identity(exp1, exp2):
    dis = np.linalg.norm(exp1-exp2, axis = 1)
    total = dis.shape[0]
    true = np.where(abs(dis)<1e-8)[0].shape[0]
    score = (total-true)/total
    return score*100

In [40]:
def calc_separability(x_test, exp_fn):
    x_test.drop_duplicates(inplace=True)
    exp = exp_fn(x_test)
    wrong = 0
    for i in range(exp.shape[0]):
        for j in range(exp.shape[0]):
            if i == j:
                continue
            eq = exp[i]==exp[j]
            if eq.all():
                wrong = wrong + 1
    total = exp.shape[0]
    score = 100*abs(wrong)/total**2
    return wrong,total,score

In [151]:
def calc_stability(exp, labels):
    total = labels.shape[0]
    label_values = np.unique(labels)
    n_clusters = label_values.shape[0]
    init = np.array([[np.average(exp[np.where(labels == i)], axis = 0)] for i in label_values]).squeeze()
    ct = sklearn.cluster.KMeans(n_clusters = n_clusters, n_jobs=5, random_state=1, n_init=10, init = init)
    ct.fit(exp)
#    a = exp[np.where(labels == 0)]
#    c_a = ct.predict(a)
#    d,f = np.unique(c_a, return_counts=True)
#    err = min(f)
#    a = exp[np.where(labels == 1)]
#    c_a = ct.predict(a)
#    d,f = np.unique(c_a, return_counts=True)
#    err = err + min(f)
    error = np.sum(np.abs(labels-ct.labels_))
    if error/total > 0.5:
        error = total-error
    return error, total

In [9]:
exp = explainer.shap_values(X_test[0:100])
exp2 = explainer.shap_values(X_test[0:100])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:25<00:00,  4.16it/s]


In [19]:
print(calc_stability(exp, y_test[0:100]))

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(29, 100)


In [18]:
i = calc_identity(exp, exp2)
print(i)

0.0


In [17]:
i = calc_identity2(exp, exp2)
print(i)

0.0


In [39]:
s = calc_separability(X_test[0:100], explainer.shap_values)
print(s)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:23<00:00,  3.98it/s]


(210, 98)


In [137]:
shap.force_plot(explainer.expected_value, exp, X_test)

In [164]:
shap.force_plot(explainer.expected_value, exp2, X_test[0:100])

In [ ]:
shap.summary_plot(exp, X_test)

In [ ]:
shap.dependence_plot("aspirin", exp, X_test)

In [166]:
shap_values_single = explainer.shap_values(X_test.iloc[1,:])

In [167]:
shap.force_plot(explainer.expected_value, shap_values_single, X_test.iloc[1,:])